# Домашнее задание 1. Части 1 и 2
## Harry Potter and the Action Prediction Challenge from Natural Language


В этом домашнем задании вы будете работать с корпусом Harry Potter and the Action Prediction Challenge. Корпус собран из фанфиков о Гарри Поттере и состоит из двух частей: 1) сырые тексты, 2) фрагменты текстов, описывающих ситуацию, в которой произнесено заклинание.

Корпус описан в статье: https://arxiv.org/pdf/1905.11037.pdf

David Vilares and Carlos Gómez-Rodríguez. Harry Potter and the Action Prediction Challenge from Natural Language. 2019 Annual Conference of the North American Chapter of the Association for Computational Linguistics. To appear.

Код для сбора корпуса находится в репозитории: https://github.com/aghie/hpac . Корпус можно скачать по инструкции из этого репозитория, но для экономии времени авторы задания уже скачали и подготовили данные к работе.

Ссылки на собранный корпус:
* Сырые тексты:  https://www.dropbox.com/s/12toaj67fjrguhd/hpac_raw.zip?dl=0
* Токенизированные тексты в нижнем регистре: https://www.dropbox.com/s/1ndto6dce5wg7j2/hpac_lower_tokenized.zip?dl=0
* train-test-dev: https://www.dropbox.com/s/ftinwwjfyydevth/hpac_splits.zip?dl=0

Части 1, 2 задания должны быть выполнены на полных текстах (сырых или предобработанных -- на ваше усмотрение), Часть 3 – на разбиение на тестовое, отладочное и обучающее множества. Тестовое множество должно быть использовано исключительно для тестирования моделей, обучающее и отладочное – для выбора модели и параметров.

В статье и репозитории вы найдете идеи, которые помогут вам выполнить домашнее задание. Их стоит воспринимать как руководство к действию, и не стоит их копировать и переиспользовать. Обученные модели использовать не нужно, код для их обучения можно использовать как подсказку.

## ПРАВИЛА
1. Домашнее задание выполняется индивидуально.
2. Домашнее задание сдается в системе Anytask, куда можно попасть через инвайт.
3. Домашнее задание оформляется в отчета в ipython-тетрадке.
4. Отчет должен содержать: нумерацию заданий и пунктов, которые вы выполнили, код решения, и понятное пошаговое описание того, что вы сделали. Отчет должен быть написан в академическом стиле, без излишнего использования сленга и с соблюдением норм русского языка.
5. Не стоит копировать фрагменты лекций, статей и Википедии в ваш отчет.
6. Плагиат и любое недобросоветсное цитирование приводит к обнуление оценки.


## Часть 1. [1 балл] Эксплоративный анализ
1. Найдите топ-1000 слов по частоте без учета стоп-слов.
2. Найдите топ-10 по частоте: имен, пар имя + фамилия, пар вида ''профессор'' + имя / фамилия.

[бонус] Постройте тематическую модель по корпусу HPAC.

[бонус] Найдите еще что-то интересное в корпусе (что-то специфичное для фанфиков или фентези-тематики)

## Часть 2. [2 балла] Модели представления слов
Обучите модель представления слов (word2vec, GloVe, fastText или любую другую) на материале корпуса HPAC.
1. Продемонстрируйте, как работает поиск синонимов, ассоциаций, лишних слов в обученной модели.
2. Визуализируйте топ-1000 слов по частоте без учета стоп-слов (п. 1.1) с помощью TSNE или UMAP (https://umap-learn.readthedocs.io).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Данные
Сырые тексты

In [ ]:
!cp /content/drive/MyDrive/hpac_raw.zip hpac_raw.zip

In [ ]:
!unzip hpac_raw.zip

Streaming output truncated to the last 5000 lines.
  inflating: fanfiction_texts/6338286  
  inflating: fanfiction_texts/6675009  
  inflating: fanfiction_texts/12615608  
  inflating: fanfiction_texts/3070076  
  inflating: fanfiction_texts/7942981  
  inflating: fanfiction_texts/4557237  
  inflating: fanfiction_texts/7646961  
  inflating: fanfiction_texts/10707993  
  inflating: fanfiction_texts/3105097  
  inflating: fanfiction_texts/8235750  
  inflating: fanfiction_texts/3334014  
  inflating: fanfiction_texts/5853244  
  inflating: fanfiction_texts/5163040  
  inflating: fanfiction_texts/4404236  
  inflating: fanfiction_texts/4059722  
  inflating: fanfiction_texts/12284676  
  inflating: fanfiction_texts/6729939  
  inflating: fanfiction_texts/7182821  
  inflating: fanfiction_texts/9080711  
  inflating: fanfiction_texts/2736446  
  inflating: fanfiction_texts/3946112  
  inflating: fanfiction_texts/7956634  
  inflating: fanfiction_texts/3174811  
  inflating: fanfiction_te

___

___

## **1. Эксплоративный анализ**

In [ ]:
import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
from tqdm.notebook import tqdm

from collections import Counter
import re

import nltk

**Найдите топ-1000 слов по частоте без учета стоп-слов:**

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# пробовала remove_stopwords из gensim, но далеко не все стоп-слова были удалены

#from gensim.parsing.preprocessing import remove_stopwords

#filtered_words = remove_stopwords(all_texts.lower())

In [ ]:
path_dir = '/content/fanfiction_texts'
path_all_texts = [os.path.join(path_dir, f) for f in os.listdir(path_dir)]

counts = Counter()

# итерируемся, чтобы не ругалось на нехватку ram, ну и так удобнее
for path_text in tqdm(path_all_texts):
    with open(path_text, 'r') as myfile:
        text = myfile.read()
        # бьем всё по словам с приведением к нижнему регистру
        words = re.findall(r'\b\w+\b', text.lower())
        for word in words:
          if word not in stop_words:
            counts.update([word])

  0%|          | 0/36225 [00:00<?, ?it/s]

In [ ]:
# 1000 самых встречаемых слов

most_common_words_1000 = counts.most_common(1000)

In [ ]:
print("Total unique tokens:", len(counts))
print('\n')
print('\n'.join(f"{word}: {count}" for word, count in most_common_words_1000[:10]))
print('...')
print('\n'.join(f"{word}: {count}" for word, count in most_common_words_1000[-10:]))

Total unique tokens: 371758


harry: 4000488
said: 2262364
hermione: 1831464
would: 1655264
back: 1400308
one: 1400055
draco: 1389966
could: 1303069
like: 1275300
know: 1203948
...
grew: 41992
act: 41951
heavy: 41948
hiding: 41928
realised: 41922
regulus: 41778
umbridge: 41764
response: 41752
shop: 41733
crossed: 41707


In [ ]:
# проверка на попадание какого-либо стоп-слова

for word, count in most_common_words_1000:
  if word == 'a':
    print(count)

**Найдите топ-10 по частоте: имен, пар имя + фамилия, пар вида ''профессор'' + имя / фамилия:**

**Список имен возьмем из интернета (например, из Википедии или Вики по Гарри Поттеру)

In [ ]:
# находим список имена/имена+фамилии персонажей

characters_names = 'Aberforth Dumbledore, Alastor Moody, Albus Dumbledore, Albus Severus Potter, Alecto Carrow, Alice Longbottom, Alicia Spinnet, Amelia Bones, Amos Diggory, Amycus Carrow, Andromeda Tonks, Angelina Johnson, Anthony Goldstein, Antioch Peverell, Antonin Dolohov, Arabella Figg, Aragog, Argus Filch, Ariana Dumbledore, Arthur Weasley, Augusta Longbottom, Augustus Rookwood, Aurora Sinistra, Bane, Barty Crouch Jr., Barty Crouch Sr., Bathilda Bagshot, Bathsheba Babbling, Beedle the Bard, Bellatrix Lestrange, Bertha Jorkins, Bill Weasley, Blaise Zabini, Buckbeak, Cadmus Peverell, Cedric Diggory, Charity Burbage, Charlie Weasley, Cho Chang, Colin Creevey, Corban Yaxley, Cormac McLaggen, Cornelius Fudge, Crookshanks, Cuthbert Binns, Dean Thomas, Dedalus Diggle, Dennis Creevey, Dilys Derwent, Dirk Cresswell, Dobby, Dolores Umbridge, Draco Malfoy, Dudley Dursley, Elphias Doge, Emmeline Vance, Ernie Macmillan, Errol, Everard, Fang, Fawkes, Fenrir Greyback, Filius Flitwick, Firenze, Fleur Delacour, Fluffy, Frank Bryce, Frank Longbottom, Fred Weasley, Fridwulfa, Gabrielle Delacour, Garrick Ollivander, Gellert Grindelwald, George Weasley, Gilderoy Lockhart, Ginny Weasley, Godric Gryffindor, Grawp, Gregorovitch, Gregory Goyle Jr., Gregory Goyle Sr., Griphook, Hannah Abbott, Harry Potter, Hedwig, Helena Ravenclaw, The Grey Lady, Helga Hufflepuff, Hepzibah Smith, Hermione Granger, Hokey, Horace Slughorn, Ignotus Peverell, Igor Karkaroff, Irma Pince, James Potter, James Sirius Potter, Justin Finch-Fletchley, Katie Bell, Kingsley Shacklebolt, Kreacher, Lavender Brown, Lee Jordan, Lily Luna Potter, Lily Potter, Lucius Malfoy, Ludo Bagman, Luna Lovegood, Mad-Eye, Mafalda Hopkirk, Magorian, Madam Malkin, Marcus Flint, Marge Dursley, Marietta Edgecombe, Marvolo Gaunt, Mary Cattermole, Mary Riddle, Merope Gaunt, Michael Corner, Millicent Bulstrode, Minerva McGonagall, Molly Weasley, Morfin Gaunt, Mundungus Fletcher, Muriel, Moaning Myrtle, Nagini, Narcissa Malfoy, Neville Longbottom, Newt Scamander, Nearly Headless Nick, Nicolas Flamel, Mrs. Norris, Nymphadora Tonks, Oliver Wood, Olympe Maxime, Padma Patil, Pansy Parkinson, Parvati Patil, Peeves, Penelope Clearwater, Percy Weasley, Peter Pettigrew, Petunia Dursley, Phineas Nigellus Black, Pigwidgeon, Pius Thicknesse, Pomona Sprout, Poppy Pomfrey, Quirinus Quirrell, Rabastan Lestrange, Reginald Cattermole, Regulus Arcturus Black, Remus Lupin, Rita Skeeter, Rodolphus Lestrange, Rolanda Hooch, Romilda Vane, Ron Weasley, Ronan, Madam Rosmerta, Rowena Ravenclaw, Rubeus Hagrid, Rufus Scrimgeour, Salazar Slytherin, Seamus Finnigan, Septima Vector, Severus Snape, Sirius Black, Stan Shunpike, Sturgis Podmore, Susan Bones, Sybill Trelawney, Patricia, Ted Tonks, Teddy Lupin, Terry Boot, Theodore Nott, Tom Riddle, Trevor, Vernon Dursley, Viktor Krum, Vincent Crabbe Jr., Vincent Crabbe Sr., Lord Voldemort, Walden Macnair, Wilhelmina Grubbly-Plank, Winky, Xenophilius Lovegood, Zacharias Smith'

In [ ]:
# top_companies, top_counts = zip(*Counter(data['Company']).most_common(1000))

In [ ]:
# создадим отдельные списки для имен, пар имя + фамилия, пар 'профессор' + имя / фамилия:

names_surnames = []
names = []
officially = []

names_split = characters_names.lower().split(sep=', ')

for item in names_split:
  if len(item.split()) > 1:
    names_surnames.append(item)
    officially.append('professor '+item)
    names.append(item.split()[0])
  else:
    names.append(item)

In [ ]:
names_surnames[:10]

['aberforth dumbledore',
 'alastor moody',
 'albus dumbledore',
 'albus severus potter',
 'alecto carrow',
 'alice longbottom',
 'alicia spinnet',
 'amelia bones',
 'amos diggory',
 'amycus carrow']

In [ ]:
officially[:10]

['professor aberforth dumbledore',
 'professor alastor moody',
 'professor albus dumbledore',
 'professor albus severus potter',
 'professor alecto carrow',
 'professor alice longbottom',
 'professor alicia spinnet',
 'professor amelia bones',
 'professor amos diggory',
 'professor amycus carrow']

In [ ]:
# ищем вхождения имен по текстам

name_counts = Counter()
name_surname_counts = Counter()
officially_counts = Counter()

for name in names:
    name_counts[name] = counts[name]

# топ-10 по частоте имен
print('Top 10 Names\n')
print('\n'.join(f"{word}: {count}" for word, count in name_counts.most_common()[:10]))

Top 10 Names

harry: 4000488
hermione: 1831464
draco: 1389966
ron: 906181
severus: 655653
ginny: 639543
sirius: 630217
james: 468947
remus: 433776
lily: 423882


In [ ]:
# для пар итерируемся по файлам заново
for path_text in tqdm(path_all_texts):
    with open(path_text, 'r') as myfile:
        text = myfile.read()
        # только приводим к нижнему регистру и производим поиск по всему тексту
        text = text.lower()
        for name in names_surnames:
            name_surname_counts[name] += text.count(name)

        for name in officially:
            officially_counts[name] += text.count(name)

# топ-10 по частоте "имя + фамилия"
print('Top 10 Names + Surnames\n')
print('\n'.join(f"{word}: {count}" for word, count in name_surname_counts.most_common()[:10]))

  0%|          | 0/36225 [00:00<?, ?it/s]

Top 10 Names + Surnames

harry potter: 130673
draco malfoy: 48368
hermione granger: 31186
severus snape: 31136
james potter: 28551
sirius black: 27286
lord voldemort: 25396
lucius malfoy: 24383
albus dumbledore: 21376
tom riddle: 18631


In [ ]:
# топ-10 по частоте "professor +имя + фамилия"
print('Top 10 Professor + Names + Surnames\n')
print('\n'.join(f"{word}: {count}" for word, count in officially_counts.most_common()[:10]))

Top 10 Professor + Names + Surnames

professor severus snape: 780
professor minerva mcgonagall: 252
professor albus dumbledore: 192
professor remus lupin: 153
professor horace slughorn: 93
professor filius flitwick: 69
professor neville longbottom: 52
professor harry potter: 45
professor dolores umbridge: 41
professor pomona sprout: 35


**Таким образом, наиболее частое имя - Гарри, что ожидаемо. Для пары имя+фамилия это так же Гарри Поттер, а вот на второй месте Драко Малфой (но я ожидала, что у них не будет настолько большой разрыв в количестве встречаемости, то есть Гарри популярнее у авторов фанфиков...)**

**Ну и самый популярный профессор - Северус Снейп. Ставим за это лайк**

## **2. Модели представления слов**

Обучение производится также на сырых данных (fanction_texts), для этого необходимо итерировать по каждому файлу для обучения, иначе будет "съеден" весь RAM. Таким образом, сначала используется итератор по файлам, а после по нему обучается Word2Vec:

In [ ]:
class WordTrainer(object):
   def __init__(self, files_path): # путь к файлам
      self.files_path = files_path
   def __iter__(self):
      for path_text in self.files_path:
          with open(path_text, 'r') as myfile:
            text = myfile.read()
            words = re.findall(r'\b\w+\b', text.lower()) # для каждого файла бьем текст по словам и приводим к нижнему регистру
            # words = [word for word in words if word not in stop_words]
          yield words # создается генератор по файлам, так у нас не сжирается много рам на одновременное использование большого количества данных

In [ ]:
token_texts = WordTrainer(path_all_texts)

In [ ]:
from gensim.models import Word2Vec

wv_model = Word2Vec(token_texts, vector_size=32, window=5, workers=4, min_count=2)

**Продемонстрируйте, как работает поиск синонимов, ассоциаций, лишних слов в обученной модели:**

In [ ]:
# поиск синонимов и ассоциаций
print('Synonyms for word:  Gold\n')
wv_model.wv.most_similar('gold')

Synonyms for word:  Gold



[('bronze', 0.9205005764961243),
 ('inlay', 0.9175809025764465),
 ('coral', 0.8976767659187317),
 ('silver', 0.8945714831352234),
 ('lime', 0.8892735242843628),
 ('rubies', 0.8880021572113037),
 ('studded', 0.8837212324142456),
 ('linings', 0.8711382746696472),
 ('neon', 0.8697117567062378),
 ('garnet', 0.869174599647522)]

In [ ]:
print('Similar for word:  Stick\n')
wv_model.wv.similar_by_vector('gold', topn=5)

Similar for word:  Stick



[('bronze', 0.9205005764961243),
 ('inlay', 0.9175809025764465),
 ('coral', 0.8976767659187317),
 ('silver', 0.8945714831352234),
 ('lime', 0.8892735242843628)]

In [ ]:
print('Associations to: as positive - Quidditch and Money, as negative - Brain\n')
wv_model.wv.most_similar(positive=["quidditch", "money"], negative=["brain"])

Associations to: as positive - Quidditch and Money, as negative - Brain



[('harpies', 0.7371070981025696),
 ('holyhead', 0.7228891849517822),
 ('soccer', 0.695338785648346),
 ('qudditch', 0.6898140907287598),
 ('chaser', 0.6883147954940796),
 ('tickets', 0.6819713115692139),
 ('quiddich', 0.6799290180206299),
 ('model', 0.6797777414321899),
 ('bulgaria', 0.6754626035690308),
 ('cannons', 0.6753119230270386)]

In [ ]:
# поиск лишнего слова
print('Redundant between :  harry, hermione, gold and ron\n')
wv_model.wv.doesnt_match(['harry','hermione','gold','ron'])

Redundant between :  harry, hermione, gold and ron



'gold'

In [ ]:
print('Redundant between :  copper, albert, gold and silver\n')
wv_model.wv.doesnt_match(['copper','albert','gold','silver'])

Redundant between :  copper, albert, gold and silver



'albert'

***Задание*: Визуализируйте топ-1000 слов по частоте без учета стоп-слов (п. 1.1) с помощью TSNE или UMAP.**

Провизуализируем с использованием TSNE. Для этого берутся значения векторов для 1000 самых встречаемых слов и "скармливаются" fit_transform.

In [ ]:
word_vectors = np.array([wv_model.wv.get_vector(x) for x, _ in most_common_words_1000]) # значения для tsne

In [ ]:
token_words = np.array([x[0] for x in most_common_words_1000]) # сами слова

In [ ]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

tsne = TSNE(n_components=2, random_state=45, n_iter=5000)
word_tsne = tsne.fit_transform(word_vectors)

scaler = StandardScaler()
word_tsne = scaler.fit_transform(word_tsne)

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_t_sne.py:1162: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


In [ ]:
# взяла функцию для построения с семинара

import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [ ]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='palevioletred', token=token_words)

figure(id='p1350', ...)

**Далее я перехожу на каггл, так как там больше GPU, чтобы выполнить 3-ий Пункт домашнего задания**